# Dense block approach

In this notebook, I share a set of functions that helped me find some optimal solutions for this competition, along with an example for 178 trees. The main idea is to first generate a dense block containing as many trees as possible, and then place the remaining trees afterward.

## Scoring functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from decimal import Decimal, getcontext

import pandas as pd
from shapely import affinity, touches
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

# Decimal precision and scaling factor
getcontext().prec = 25
scale_factor = Decimal('1e18')

class ParticipantVisibleError(Exception):
    pass

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)
        scale_factor = Decimal('1e18')
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon(
            [
                # Start at Tip
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                # Right side - Top Tier
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                # Right side - Middle Tier
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                # Right side - Bottom Tier
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                # Right Trunk
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                # Left Trunk
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Bottom Tier
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Middle Tier
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                # Left side - Top Tier
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))


def score(submission: pd.DataFrame) -> float:
    """
    For each n-tree configuration, the metric calculates the bounding square
    volume divided by n, summed across all configurations.

    This metric uses shapely v2.1.2.

    Examples
    -------
    >>> import pandas as pd
    >>> row_id_column_name = 'id'
    >>> data = [['002_0', 's-0.2', 's-0.3', 's335'], ['002_1', 's0.49', 's0.21', 's155']]
    >>> submission = pd.DataFrame(columns=['id', 'x', 'y', 'deg'], data=data)
    >>> solution = submission[['id']].copy()
    >>> score(solution, submission, row_id_column_name)
    0.877038143325...
    """
    scale_factor = Decimal('1e18')
    # remove the leading 's' from submissions
    data_cols = ['x', 'y', 'deg']
    submission = submission.astype(str)
    for c in data_cols:
        if not submission[c].str.startswith('s').all():
            raise ParticipantVisibleError(f'Value(s) in column {c} found without `s` prefix.')
        submission[c] = submission[c].str[1:]

    # enforce value limits
    limit = 100
    bad_x = (submission['x'].astype(float) < -limit).any() or \
            (submission['x'].astype(float) > limit).any()
    bad_y = (submission['y'].astype(float) < -limit).any() or \
            (submission['y'].astype(float) > limit).any()
    if bad_x or bad_y:
        raise ParticipantVisibleError('x and/or y values outside the bounds of -100 to 100.')

    # grouping puzzles to score
    submission['tree_count_group'] = submission['id'].str.split('_').str[0]

    total_score = Decimal('0.0')
    for group, df_group in submission.groupby('tree_count_group'):
        num_trees = len(df_group)

        # Create tree objects from the submission values
        placed_trees = []
        for _, row in df_group.iterrows():
#             print(row['x'], row['y'], row['deg'])
            placed_trees.append(ChristmasTree(row['x'], row['y'], row['deg']))

        # Check for collisions using neighborhood search
        all_polygons = [p.polygon for p in placed_trees]
        r_tree = STRtree(all_polygons)

        # Checking for collisions
        max_inter = 0
        for i, poly in enumerate(all_polygons):
            indices = r_tree.query(poly)
            for index in indices:
                if index == i:  # don't check against self
                    continue
                if poly.intersects(all_polygons[index]) and not poly.touches(all_polygons[index]):

                    print(poly)
                    raise ParticipantVisibleError(f'Overlapping trees in group {group}')

        # Calculate score for the group
        bounds = unary_union(all_polygons).bounds
        # Use the largest edge of the bounding rectangle to make a square boulding box
        side_length_scaled = max(bounds[2] - bounds[0], bounds[3] - bounds[1])

        group_score = (Decimal(side_length_scaled) ** 2) / (scale_factor**2) / Decimal(num_trees)
        total_score += group_score

    return float(total_score)

def draw(submission):
    scale_factor = Decimal('1e18')
    # remove the leading 's' from submissions
    data_cols = ['x', 'y', 'deg']
    submission = submission.astype(str)
    for c in data_cols:
        if not submission[c].str.startswith('s').all():
            raise ParticipantVisibleError(f'Value(s) in column {c} found without `s` prefix.')
        submission[c] = submission[c].str[1:]

    # enforce value limits
    limit = 100
    bad_x = (submission['x'].astype(float) < -limit).any() or \
            (submission['x'].astype(float) > limit).any()
    bad_y = (submission['y'].astype(float) < -limit).any() or \
            (submission['y'].astype(float) > limit).any()
    if bad_x or bad_y:
        raise ParticipantVisibleError('x and/or y values outside the bounds of -100 to 100.')

    # grouping puzzles to score
    submission['tree_count_group'] = submission['id'].str.split('_').str[0]

    total_score = Decimal('0.0')
    for group, df_group in submission.groupby('tree_count_group'):
        num_trees = len(df_group)

        # Create tree objects from the submission values
        placed_trees = []
        for _, row in df_group.iterrows():
#             print(row['x'], row['y'], row['deg'])
            placed_trees.append(ChristmasTree(row['x'], row['y'], row['deg']))

        # Check for collisions using neighborhood search
        all_polygons = [p.polygon for p in placed_trees]
        r_tree = STRtree(all_polygons)

        # Checking for collisions
        max_inter = 0
        for i, poly in enumerate(all_polygons):
            xx,yy = poly.exterior.xy
            plt.gca().set_aspect('equal')
            plt.plot(np.array(xx)/1e18,np.array(yy)/1e18)
            indices = r_tree.query(poly)
            for index in indices:
                if index == i:  # don't check against self
                    continue
                if poly.intersects(all_polygons[index]) and not poly.touches(all_polygons[index]):
                    max_inter = max(max_inter, poly.intersection(all_polygons[index]).area/2.4562499999999998e+35)
        print('Intersection = ', max_inter)

        # Calculate score for the group
        bounds = unary_union(all_polygons).bounds
        # Use the largest edge of the bounding rectangle to make a square boulding box
        side_length_scaled = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
        print('Width = ', bounds[2] - bounds[0], 'Height = ', bounds[3] - bounds[1])
        group_score = (Decimal(side_length_scaled) ** 2) / (scale_factor**2) / Decimal(num_trees)
        total_score += group_score

    print('Score = ', float(total_score))

## Dense block functions

We have two dense-block functions: **gen_dense_block1**, which depends on four parameters (rows, columns, angle, and pairwise distance), and **gen_dense_block2**, which depends on three parameters (rows, columns, and angle). All other parameters that are optimal for each block can be determined through a simple optimization procedure in just a few seconds.

In [ ]:
def gen_block(x_len, y_len, deg1, deg2, shift_x1, shift_y1, shift_x2, shift_y, sign):
    x = 0
    y0 = 0
    eps = 0.00000000000001
    data = []
    k = 0
    for i in range(x_len):
        if i % 2 == 1:
            angle = deg2
            x += shift_x1 + eps
            y0 += shift_y1 + eps
        else:
            angle = deg1
            x += shift_x2 + eps
            y0 += -shift_y1 - eps * sign
        y = y0
        for j in range(y_len):
            y += shift_y + eps
            data.append(['_'+str(k), 's' + str(x), 's' + str(y), 's' + str(angle)])
            k = k + 1
    return pd.DataFrame(columns=['id', 'x', 'y', 'deg'], data=data)

def transpose(df):
    x = df.x.copy()
    df.x = df.y.str[0] + (-df.y.str[1:].astype(float)).astype(str)
    df.y = x
    df.deg = df.deg.str[0] + (df.deg.str[1:].astype(float)+90).astype(str)
    return df

def find_shift_x1(deg, shift_y1):
    def fun_min(shift_x1_list):
        tree1 = ChristmasTree(shift_x1_list[0], shift_y1, deg - 180)
        if tree1.polygon.intersects(tree0.polygon):
            return 1000
        if tree1.polygon.bounds[2] < tree0.polygon.bounds[0] + 0.3 * 1e18:
            return 1000
        return shift_x1_list[0]

    from scipy.optimize import minimize
    tree0 = ChristmasTree(0, 0, deg)
    res = minimize(fun_min, [20], method='Powell')
    return res.x[0]

def find_shift_y1(deg, shift_x1):
    def fun_min(shift_y1_list):
        tree1 = ChristmasTree(shift_x1, shift_y1_list[0], deg - 180)
        if tree1.polygon.intersects(tree0.polygon):
            return 1000
        if tree1.polygon.bounds[3] < tree0.polygon.bounds[1] + 0.3 * 1e18:
            return 1000
        return shift_y1_list[0]

    from scipy.optimize import minimize
    tree0 = ChristmasTree(0, 0, deg)
    res = minimize(fun_min, [20], method='Powell')
    return res.x[0]

def find_shift_x2(deg, shift_x1, shift_y1):
    def fun_min(shift_x2_list):
        tree10 = ChristmasTree(shift_x1 + shift_x2_list[0], 0, deg)
        tree11 = ChristmasTree(shift_x1 * 2 + shift_x2_list[0], shift_y1, deg - 180)
        pair1 = unary_union([tree10.polygon, tree11.polygon])
        if pair1.intersects(pair0):
            return 1000
        if pair1.bounds[2] < pair0.bounds[0] + 0.3 * 1e18:
            return 1000
        return shift_x2_list[0]

    from scipy.optimize import minimize
    tree00 = ChristmasTree(0, 0, deg)
    tree01 = ChristmasTree(shift_x1, shift_y1, deg - 180)
    pair0 = unary_union([tree00.polygon, tree01.polygon])
    res = minimize(fun_min, [20], method='Powell')
    return res.x[0]

def find_shift_y2(deg, shift_x1, shift_y1, shift_x2):
    def fun_min(shift_y2_list):
        tree10 = ChristmasTree(0, shift_y2_list[0], deg)
        tree11 = ChristmasTree(shift_x1, shift_y1 + shift_y2_list[0], deg - 180)
        tree12 = ChristmasTree(shift_x1 + shift_x2, shift_y2_list[0], deg)
        tree13 = ChristmasTree(shift_x1 * 2 + shift_x2, shift_y1 + shift_y2_list[0], deg - 180)
        layer1 = unary_union([tree10.polygon, tree11.polygon, tree12.polygon, tree13.polygon])
        if layer1.intersects(layer0):
            return 1000
        if layer1.bounds[3] < layer0.bounds[1] + 0.3 * 1e18:
            return 1000
        return shift_y2_list[0]

    from scipy.optimize import minimize
    tree00 = ChristmasTree(0, 0, deg)
    tree01 = ChristmasTree(shift_x1, shift_y1, deg - 180)
    tree02 = ChristmasTree(shift_x1 + shift_x2, 0, deg)
    tree03 = ChristmasTree(shift_x1 * 2 + shift_x2, shift_y1, deg - 180)
    layer0 = unary_union([tree00.polygon, tree01.polygon, tree02.polygon, tree03.polygon])
    res = minimize(fun_min, [20], method='Powell')
    return res.x[0]

def gen_dense_block1(x_len, y_len, deg, d):
    shift_x1 = np.abs(d * np.sin(deg * np.pi / 360))
    shift_y1 = find_shift_y1(deg, shift_x1)
    shift_x2 = find_shift_x2(deg, shift_x1, shift_y1)
    shift_y2 = find_shift_y2(deg, shift_x1, shift_y1, shift_x2)
    return gen_block(x_len, y_len, deg, deg-180, shift_x1, shift_y1, shift_x2, shift_y2, 1)

def gen_dense_block2(x_len, y_len, deg, up = True):
    sign = 1 - 2 * up
    alpha = 3 * np.pi / 2 - deg / 180 * np.pi
    shift_y1 = sign * 0.15 * np.cos(alpha) + 0.2 * np.sin(alpha)
    shift_x1 = find_shift_x1(deg, shift_y1)
    shift_x2 = -sign * 0.15 * np.sin(alpha) + 0.2 * np.cos(alpha)
    shift_y2 = find_shift_y2(deg, shift_x1, shift_y1, shift_x2)
    return gen_block(x_len, y_len, deg, deg-180, shift_x1, shift_y1, shift_x2, shift_y2, sign)

## Examples

This is an example of use of the **gen_dense_block1** function. The angle controls the orientation of the bottom-left tree in the block, while distance defines the horizontal x-offset between trees in pair if they alo were horizontal. From these initial parameters, the relative positions of trees, horizontal inter-pair spacing, and vertical layer spacing are obtained via a simple optimization procedure that completes in a few seconds. The current values: *248.19859051364818, 1.1014707194584321* are near a local optimum, but can be tuned to favour width over height or vice versa.

In [ ]:
draw(gen_dense_block1(12, 14, 248.19859051364818, 1.1014707194584321))

This is an example of the **gen_dense_block2** function. The angle parameter specifies the orientation of the bottom-left tree in the block. All parameters within each pair are chosen so that the tree stumps align exactly, with the stump of the right pair positioned directly beneath the stump of the left pair. All remaining parameters are determined in the same way as in the previous case. As before, the angle can be adjusted to trade off block width against height, or vice versa.

In [ ]:
draw(gen_dense_block2(12, 14, 259, False))

This case is similar to the previous one, except that the stump of the right pair is positioned above the stump of the left pair.

In [ ]:
draw(gen_dense_block2(12, 14, 250, True))

If you need a dense vertical block, it is often easier to obtain it by transposing the layout.

In [ ]:
draw(transpose(gen_dense_block2(12, 1, 259, False)))

## 178 trees

Here is an example of an optimal solution. The current optimal solution for 178 trees has both width and height equal to 7.715. Our dense block of 12 × 14 = 168 trees has a height of 7.714 and a width of 7.12. Therefore, if we can place the remaining 10 trees to the right of the dense block—within the horizontal range from 7.12 to 7.714—we can further improve the solution.

This can be done in several ways, such as using an interactive editor, applying a bbox3-style optimization, generating an additional vertical block, or similar techniques. In this particular case, I achieved the improvement simply by using an interactive editor.

In [ ]:
df = gen_dense_block1(12, 14, 248.19859051364818, 1.1014707194584321)
df.loc[168] = ['_168','s7.378704','s7.676734','s167.8']
df.loc[169] = ['_169','s7.403704','s6.690123','s338.2']
df.loc[170] = ['_170','s7.403704','s5.288231','s338.2']
df.loc[171] = ['_171','s7.368704','s6.252972','s167.8']
df.loc[172] = ['_172','s7.403704','s3.664810','s338.2']
df.loc[173] = ['_173','s7.368704','s4.629408','s167.8']
df.loc[174] = ['_174','s7.403704','s2.065766','s338.2']
df.loc[175] = ['_175','s7.358704','s3.026393','s167.8']
df.loc[176] = ['_176','s7.403704','s0.434170','s338.2']
df.loc[177] = ['_177','s7.368704','s1.399315','s167.8']
df.id = df.id.str[2:1] + '178' + df.id.str[:]
draw(df)

We can now compute the score before and after the improvement and save the resulting solution.

In [ ]:
df_all = pd.read_csv('/kaggle/input/santa2025/submission.csv')
score(df_all)

In [ ]:
df_all.loc[df_all.id.str[:3] == '178'] = df.values
score(df_all)

In [ ]:
df_all.to_csv('submission.csv', index=False)